# Preprocessing

In [111]:
import os
import sys
import numpy as np
import gensim

In [112]:
file_path = os.path.join('review_data','ratings.txt')

In [113]:
sentences = []
labels = []
with open(file_path, 'r') as f:
    next(f) # header skip
    for line in f.readlines():
        _, doc, label = line.strip().split('\t')
        sentences.append(doc.strip())
        labels.append(label.strip())

In [114]:
len(sentences) # 전체 문장

200000

In [115]:
sentences[:5]

['어릴때보고 지금다시봐도 재밌어요ㅋㅋ',
 '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.',
 '폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.',
 '와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지',
 '안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.']

In [116]:
from konlpy.tag import Twitter
twi_tagger = Twitter()

In [117]:
# 품사 중 명사, 동사, 형용사, 부사, 감탄사(헐, 어머나), 한국어약어(ㅋㅋㅋ) 만 추출해봄.
Non_Stop_words = set(["Noun", "Verb","Adjective","Adverb","Exclamation","KoreanParticle"])

# (돈, Noun) -> "돈/Noun" 형태로 형태소 분석.
def tokenizer_twit(doc, remove_stopwords=True):
    if remove_stopwords:
        word_list = ['/'.join(t) for t in twi_tagger.pos(doc, norm=True, stem=True) if t[-1] in Non_Stop_words]
    else:
        word_list = ['/'.join(t) for t in twi_tagger.pos(doc, norm=True, stem=True)]
    return word_list

def tokenizer(sentence):
    tokens = re.findall(r"[\w]+|[^\s\w]", sentence)
    return tokens

In [118]:
tokenizer(sentences[1])

['디자인을',
 '배우는',
 '학생으로',
 ',',
 '외국디자이너와',
 '그들이',
 '일군',
 '전통을',
 '통해',
 '발전해가는',
 '문화산업이',
 '부러웠는데',
 '.',
 '사실',
 '우리나라에서도',
 '그',
 '어려운시절에',
 '끝까지',
 '열정을',
 '지킨',
 '노라노',
 '같은',
 '전통이있어',
 '저와',
 '같은',
 '사람들이',
 '꿈을',
 '꾸고',
 '이뤄나갈',
 '수',
 '있다는',
 '것에',
 '감사합니다',
 '.']

In [119]:
tokenizer_twit(sentences[1], remove_stopwords=True)

['디자인/Noun',
 '배우다/Verb',
 '학생/Noun',
 '외국/Noun',
 '디자이너/Noun',
 '그/Noun',
 '일군/Noun',
 '전통/Noun',
 '통해/Noun',
 '발전/Noun',
 '하다/Verb',
 '문화/Noun',
 '산업/Noun',
 '부럽다/Adjective',
 '사실/Noun',
 '우리나라/Noun',
 '그/Noun',
 '어렵다/Adjective',
 '시절/Noun',
 '끝/Noun',
 '열정/Noun',
 '지키다/Verb',
 '노라노/Noun',
 '같다/Adjective',
 '전통/Noun',
 '있다/Adjective',
 '저/Noun',
 '같다/Adjective',
 '사람/Noun',
 '꿈/Noun',
 '꾸다/Verb',
 '이루다/Verb',
 '나가다/Verb',
 '수/Noun',
 '있다/Adjective',
 '것/Noun',
 '감사/Noun',
 '하다/Verb']

In [ ]:
words_list=[]
statement_num=0
for sent in sentences:
    words_list.append(tokenizer_twit(sent, remove_stopwords=False))
#     words_list.append(tokenizer(sent))
    statement_num += 1
    if statement_num%10000 == 0:
        print("statement_num : %d" % statement_num)

statement_num : 10000
statement_num : 20000


# Word2vec Modeling

In [ ]:
from gensim.models import word2vec
import multiprocessing
import time
from sklearn.manifold import TSNE
from bokeh.plotting import figure, show, save, output_file
from bokeh.models import ColumnDataSource, LabelSet
from bokeh import palettes

In [ ]:
# Set parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 10   # Minimum word count                        
num_workers = multiprocessing.cpu_count() # Number of threads to run in parallel
context = 3 # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
negative = 20
iter_ = 20 # 얼마나 반복할지.
sg=1

In [ ]:
start = time.time() # 작업시간 검사용!
# Initialize and train the model (this will take some time)
print("Training model...")
model = word2vec.Word2Vec(words_list, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, negative=negative, iter=iter_, sg=sg)
end = time.time()
print(end-start)

In [ ]:
len(model.wv.index2word) # 전체 단어

In [ ]:
# 모델 저장
model_name = os.path.join('w2v_file','300features_10minwords_3context_twi2')
model.save(model_name)

In [ ]:
# # 모델 로드.
# model_name = os.path.join('w2v_file','300features_10minwords_3context_twi')
# model = word2vec.Word2Vec.load(model_name)

In [ ]:
def vis_top_n(model, filepath, top, lr=500, n_iter=1000, perplexity=10):
    """
    전체 단어 중 특정 빈도 이상의 단어만 시각화.
    :param model: gensim word2vec model
    :param filepath: 시각화파일(html) 저장할 파일 경로
    :param top: 빈도 상위 몇개의 단어까지 시각화할지
    """
    vectors = model.wv.syn0[:top]
    labels = model.wv.index2word[:top]
    counts = [model.wv.vocab[label].count for label in labels]  # _labels의 빈도수 (빈도수별 사이즈 다르게 하기 위해)

    if np.shape(vectors)[1] > 2:  # 입력값이 2차원일경우에는 차원축소 하지 않음.
        print('tsne....')
        start = time.time()
        tsne = TSNE(perplexity=perplexity, n_components=2, init='random', n_iter=n_iter, verbose=1, learning_rate=lr, method='exact')  # tsne를 이용한 차원 축소 (n차원 -> 2차원)
        vectors = tsne.fit_transform(vectors)
        print(time.time()-start)

    _filepath = '%s[top_%i].html' % (filepath, top)

    source = ColumnDataSource(
        data=dict(
            x=vectors.T[0],
            y=vectors.T[1],
            size=(np.log1p(np.array(counts))) * 1.2 + 2,
            word=labels,
            color=['#0099ff'] * len(labels),
        )
    )
    label_set = LabelSet(x='x', y='y', text='word', level='glyph', text_color="#111111", text_alpha=0.6, text_font_size='8pt', x_offset=5, y_offset=5, source=source, render_mode='canvas')
    tools = "pan,wheel_zoom,box_zoom,reset,resize"
    p = figure(plot_width=900, plot_height=900, tools=[tools], title='word2vec vis top %i' % top)
    p.circle('x', 'y', size='size', source=source, alpha=0.6, fill_color='color', line_color='#eeeeee')
    p.add_layout(label_set)

    output_file(_filepath, title=_filepath)
    show(p)

In [ ]:
# vis_top_n(model=model, filepath=os.path.join('w2v_file','300features_10minwords_3context_twi'), top=2000, lr=500, n_iter=1000, perplexity=10)

# CNN

In [ ]:
from bs4 import BeautifulSoup

import re
import sys
import os
import numpy as np
import pandas as pd
from nltk import tokenize

from sklearn.metrics import classification_report

In [ ]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras import losses

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout
from keras.models import Model

In [ ]:
words_list[:5]

In [ ]:
labels[:5]

In [ ]:
w2v_dics = set(model.wv.index2word)

In [ ]:
model.wv.index2word

In [ ]:
sequences = [[model.wv.index2word.index(word) for word in wl if word in model.wv.index2word] for wl in words_list]

In [ ]:
sequences[:10]

In [ ]:
MAX_SEQUENCE_LENGTH = 50
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH) # MAX_SEQUENCE_LENGTH만큼 뒤 기준으로 잘림.

In [ ]:
labels = to_categorical(np.asarray(labels)) # label one-hot encoding
labels

In [ ]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

In [ ]:
indices = np.arange(data.shape[0])
indices[:20]

In [ ]:
np.random.shuffle(indices) # 데이터 셔플링

In [ ]:
indices[:20]

In [ ]:
data = data[indices] # 데이터 셔플링
labels = labels[indices]

In [ ]:
VALIDATION_SPLIT = 0.3
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
X_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
print('Number of positive and negative reviews in traing and validation set ')
print(y_train.sum(axis=0))
print(y_test.sum(axis=0))

In [ ]:
EMBEDDING_DIM = 300

In [ ]:
embedding_matrix = np.random.random((len(model.wv.index2word) + 1, EMBEDDING_DIM)) # bias term 때문에 +1 해준듯.. 
embedding_matrix # 랜덤값으로 초기화

In [ ]:
## TODO : 랜덤값 -1~1로 하기

In [ ]:
embedding_matrix.shape

In [ ]:
for i, word in enumerate(model.wv.index2word):
    embedding_matrix[i] = model[word]

## 1) A simplified Convolutional
- Simply use total 128 filters with size 5 and max pooling of 5 and 35

In [ ]:
embedding_layer = Embedding(len(model.wv.index2word) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [ ]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
# l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
# l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool2)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(2, activation='softmax')(l_dense)

In [ ]:
model = Model(sequence_input, preds)
model.compile(loss=losses.categorical_crossentropy, optimizer='rmsprop', metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=4096)

In [ ]:
# preds = model.predict_classes(X_test, batch_size=4096)

In [ ]:
preds = model.predict(X_test, batch_size=1024)
preds = [1 if p[1]>p[0] else 0 for p in preds]

In [ ]:
actuals = [1 if a[1]>a[0] else 0 for a in y_test]

In [ ]:
print('* 정확도 : %.5f' % (np.sum(np.array(actuals) == np.array(preds)) / float(len(actuals))))

In [ ]:
grade = [0,1]

In [ ]:
preds = pd.Categorical(preds, categories=grade)
actuals = pd.Categorical(actuals, categories=grade)

In [ ]:
print(pd.crosstab(actuals, preds, rownames=['actuals'], colnames=['preds']))

In [ ]:
print(classification_report(actuals, preds))